In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
# from langchain_community import embeddings
from langchain_community.embeddings import OllamaEmbeddings
# from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

In [4]:
model_local = ChatOllama(model="mistral",temperature=0)
# set_llm_cache(InMemoryCache)


In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\patel\Desktop\Final_Ms_chatbot\chatbot\4_extra_ms_new_ppt_content.pdf")
pages = loader.load()

In [8]:
pages

[Document(page_content='aCD-20 Therapies in \nMultiple Sclerosis\nM-IN-00003371', metadata={'source': 'C:\\Users\\patel\\Desktop\\Final_Ms_chatbot\\chatbot\\4_extra_ms_new_ppt_content.pdf', 'page': 0}),
 Document(page_content='2\nDisclaimers\nThe Content in this presentation is intended for healthcare professionals in India only. \nThe medical information in this presentation is provided as an information resource \nonly and is not to be used or relied on for any diagnostic or treatment purposes\nRoche India does not promote or support the use of medications manufactured by it \nin off label indications. For specific information regarding various therapeutic agents, \nincluding Roche products, please refer to the approved full prescribing information', metadata={'source': 'C:\\Users\\patel\\Desktop\\Final_Ms_chatbot\\chatbot\\4_extra_ms_new_ppt_content.pdf', 'page': 1}),
 Document(page_content='3\nLYMPH \nNODEBLOOD \nVESSELBLOOD –\nBRAIN\nBARRIERCENTRAL NERVOUS SYSTEM\nB \nCELLT \nCELL

In [9]:
len(pages)

97

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter_rec = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex=False
)
rec_texts = text_splitter_rec.split_documents(pages)
rec_vectorstore = FAISS.from_documents(documents=rec_texts, embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'))
rec_vectorstore.save_local("4_new_ppt_new_rec")

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)

doc_splits = text_splitter.split_documents(pages)
char_vectorstore = FAISS.from_documents(documents=doc_splits, embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'))
char_vectorstore.save_local("4_new_ppt_new_char")

In [3]:
embedding = embeddings.OllamaEmbeddings(model='nomic-embed-text')

db1 = FAISS.load_local("ppt_new_rec", embeddings=embedding,allow_dangerous_deserialization=True)
db2 = FAISS.load_local("4_new_ppt_new_rec", embeddings=embedding,allow_dangerous_deserialization=True)
db1.merge_from(db2)
db1.save_local("new_multiple_sclerosis_faiss")


In [5]:
embedding = embeddings.OllamaEmbeddings(model='nomic-embed-text')

# db=FAISS.load_local("ppt_new_rec", embeddings=embedding,allow_dangerous_deserialization=True)
# db=FAISS.load_local("new_multiple_sclerosis_faiss", embeddings=embedding,allow_dangerous_deserialization=True)
db=FAISS.load_local("mahin_ppt_rec", embeddings=embedding,allow_dangerous_deserialization=True)

In [10]:
query = "Recently I was diagnosed with Relapsing-onset MS and the indicated MS phenotype is Highly active. List the name of the drugs which can be used to treat my condition"
docs = db.similarity_search(query)
print(docs[0].page_content)

6
MS is a heterogeneous disease, conventionally defined in the clinical 
setting as either relapsing or progressive phenotypes
SPMSDisabilityCIS
TimePPMSProgressive forms of MS1,3
RRMS
Time Time TimeRelapsing forms of MS (RMS)1,2Based on the natural history of the disease, MS is typically described as having one of four phenotypes:
MS phenotypes are further subcategorised by disease activity (clinically or by MRI)
and are deemed to be either ‘active’ or ‘non -active’
CIS, clinically isolated syndrome; PPMS, primary progressive MS; RRMS, relapsing remitting MS; SPMS, secondary progressive MS .
1. Lublin FD, et al. Neurology 2014;83:278 –86; 
2. National Multiple Sclerosis Society. Relapse -remitting MS (RRMS). http://www.nationalmssociety.org/What -is-MS/Types -of-MS/Relapsing -remitting -MS. Accessed January 
2020; 
3. National Multiple Sclerosis Society. Primary progressive MS (PPMS). http://www.nationalmssociety.org/What -is-MS/Types -of-MS/Primary -progressive -MS. Accessed January


In [4]:
query = "Recently I was diagnosed with Relapsing-onset MS and the indicated MS phenotype is Highly active. List the name of the drugs which can be used to treat my condition"
docs = db.similarity_search(query)
print(docs[0].page_content)

22
Where do DMTs in MS fit in the current treatment paradigm?
•Alemtuzumab*
•Cladribine
•Fingolimod
•Natalizumab†
•Ocrelizumab
•Ofatumumab3Indicated MS phenotype Approved DMTs1stattack 
suggestive 
of MS
•Glatiramer acetate
•Interferon β
•Ponesimod1,2CIS
Inactive
No clinical 
attacks and 
stable MRIActive
1 relapse in the 
previous year and/or 
new T2 or Gd+ 
lesions on MRIHighly -active
With DMT: 1 relapse and new 
Gd+ lesion(s) and/or significant 
increase in T2 lesions
No DMT: ≥2 relapses in the 
previous year and MRI
Active 
monitoring 
(clinical 
and MRI)•Dimethyl fumarate
•Fingolimod
•Glatiramer acetate
•Interferon β
•Ocrelizumab
•Teriflunomide
•Ozanimod
•Ofatumumab3
•Ponesimod1,2Relapsing -onset MS
SPMS
Progressive increase in 
disability independent of 
relapses following an 
initial relapsing coursePPMS
Progressive increase in 
disability independent of 
relapses from onset
•Siponimod
(Active SPMS)
•Ponesimod1,2
(Active SPMS)
•Ocrelizumab
•Ofatumumab3Ocrelizumab


In [6]:
retriever = db.as_retriever(search_kwargs={'k':5})

In [7]:
# after_rag_template = """
# Answer the following question only based on the provided context.
#                     Think step by step before providing a detailed answer.Provide only the answer and don't be expressive.Please don't start your answer with 'based on the provided context'.Please don't start your answer with 'According to the text'.
#                     understand the whole given context and then provide the appropriate answer for the question
#                     <context>{context}</context>
#                     Question:{question}
# """
after_rag_template = """
Please respond to the question using the context provided. Take a moment to understand the context    fully before crafting a detailed response. Your answer should be direct and concise without any expressive language. Avoid starting your response with phrases like 'based on the provided context' or 'According to the text' or 'The context does not directly answer your question'. If the question is not relevant to the context, simply respond with 'I am a Chatbot designed to assist you with inquiries about Multiple Sclerosis. Feel free to ask any questions related to this topic'.Don't provide any references in your answer even though it is provided in the given context.  
        <context>{context}</context>
        Question:{question}
"""

In [9]:
model = ChatOllama(model="mistral",temperature=0)
set_llm_cache(InMemoryCache)


In [10]:
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | model
        | StrOutputParser()
    )

In [11]:
async for chunk in after_rag_chain.astream("What are the Patient driven factors that HCPs should consider before starting the treatment for MS"):
    print(chunk, end="", flush=True)

 The patient-driven factors that HCPs should consider before starting MS treatment include disease activity and prognosis, non-response to treatment, acceptance or aversion to risk, individual risk profile (e.g. JCV serostatus), pregnancy and family planning, preference for route / lifestyle, adherence / treatment engagement.

In [12]:
async for chunk in after_rag_chain.astream("What are the Patient driven factors that HCPs should consider before starting the treatment for MS"):
    print(chunk, end="", flush=True)

 The patient-driven factors that HCPs should consider before starting MS treatment include disease activity and prognosis, non-response to treatment, acceptance or aversion to risk, individual risk profile (e.g. JCV serostatus), pregnancy and family planning, preference for route / lifestyle, adherence / treatment engagement.

In [15]:
async for chunk in after_rag_chain.astream("What are the Patient driven factors that HCPs should consider before starting the treatment for MS"):
    print(chunk, end="", flush=True)

 The patient-driven factors that HCPs should consider before starting MS treatment include disease activity and prognosis, non-response to treatment, acceptance or aversion to risk, individual risk profile (e.g. JCV serostatus), pregnancy and family planning, preference for route / lifestyle, adherence / treatment engagement.

In [9]:
async for chunk in after_rag_chain.astream("What are the Patient driven factors that HCPs should consider before starting the treatment for MS"):
    print(chunk, end="", flush=True)

 Patient-driven factors that HCPs should consider before starting MS treatment include disease activity and prognosis, non-response to treatment, acceptance of/aversion to risk, individual risk profile (e.

In [9]:
async for chunk in after_rag_chain.astream("What are the Patient driven factors that HCPs should consider before starting the treatment for MS"):
    print(chunk, end="", flush=True)

 Patient-driven factors that HCPs should consider before starting MS treatment include disease activity and prognosis, non-response to treatment, acceptance of/aversion to risk, individual risk profile (e.g. JCV serostatus), pregnancy and family planning, preference for route / lifestyle, adherence / treatment engagement.

In [ ]:
async for chunk in after_rag_chain.astream("Recently I was diagnosed with Relapsing-onset MS and the indicated MS phenotype is Highly active. List the name of the drugs which can be used to treat my condition"):
    print(chunk, end="", flush=True)

In [10]:
async for chunk in after_rag_chain.astream("Recently I was diagnosed with Relapsing-onset MS and the indicated MS phenotype is Highly active. List the name of the drugs which can be used to treat my condition"):
    print(chunk, end="", flush=True)

 Based on the information provided in the documents, the following disease-modifying therapies (DMTs) are indicated for treating highly active Relapsing-onset MS:

1. Alemtuzumab
2. Cladribine
3. Fingolimod
4. Natalizumab (†)
5. Ocrelizumab
6. Ofatumumab
7. Siponimod (Active SPMS)
8. Ponesimod (Active SPMS)

(†) Note that Natalizumab is also indicated for treating relapsing-remitting MS, but it's important to consult your healthcare provider for the most accurate information regarding your specific case.

In [11]:
async for chunk in after_rag_chain.astream("RRMS is equally common in women and men right?"):
    print(chunk, end="", flush=True)

 Based on the provided context, RRMS has a higher incidence in females (3:1 ratio, female to male).

In [12]:
async for chunk in after_rag_chain.astream("What gives rise to the abnormal immunological response seen in MS?"):
    print(chunk, end="", flush=True)

 The abnormal immunological response in MS is given rise by a subset of autoreactive B cells that bypass immune tolerance mechanisms and mature into pathogenic effectors within the central nervous system (CNS). These B cells contribute to MS pathogenesis through various mechanisms, including antigen presentation, antibody production, cytokine regulation, and formation of lymphoid structures.

In [13]:
async for chunk in after_rag_chain.astream("driving factors in MS treatment protocol"):
    print(chunk, end="", flush=True)

 The driving factors in MS treatment decisions include patient-specific factors such as disease activity and prognosis, non-response to treatment, acceptance or aversion to risk, individual risk profile (e.g. JCV serostatus), pregnancy and family planning, preference for route / lifestyle, adherence / treatment engagement; and treatment-specific factors such as efficacy profile, approved label, safety and tolerability burden, immune cell effects / reversibility, route and dose burden, monitoring requirements, and economic and resource burden. (from the context of the provided documents)

In [14]:
async for chunk in after_rag_chain.astream("what is ppms and treatment options available"):
    print(chunk, end="", flush=True)

 Primary Progressive Multiple Sclerosis (PPMS) is a type of multiple sclerosis (MS) characterized by a steady worsening of the disease from its onset, without distinct relapses or remissions. The treatment options for PPMS include:

1. Disease-modifying therapies (DMTs): These medications are designed to reduce the frequency and severity of MS relapses and slow down the progression of the disease. However, their effectiveness in PPMS is less clear compared to relapsing-remitting MS. Some DMTs that have been studied in PPMS include:
   * Dimethyl fumarate (Tecfidera)
   * Fingolimod (Gilenya)
   * Teriflunomide (Aubagio)
   * Natalizumab (Tysabri)
   * Mitoxantrone (Novantrone)
   * Rituximab
2. Ocrelizumab: This is a monoclonal antibody that has been approved for the treatment of PPMS based on its efficacy in slowing down disease progression as shown in clinical trials such as ORATORIO.
3. Symptomatic treatments: These medications are used to manage specific symptoms of MS, such as pai

In [16]:
async for chunk in after_rag_chain.astream("what are the approches in management of MS"):
    print(chunk, end="", flush=True)

 The approaches in management of Multiple Sclerosis (MS) are escalation approach and induction approach.

The escalation approach starts with lower-risk, lower-efficacy Disease-Modifying Therapies (DMTs) and moves onto more aggressive treatments if the ongoing approach fails. This approach is based on concerns over the complex safety profile of higher-efficacy DMTs and reserves high-efficacy DMTs for patients at greatest risk of accumulating disability.

The induction approach, on the other hand, is based on the underlying pathophysiology of MS and targets initial inflammatory phases to help reduce subsequent chronic accumulation of damage. This aims to reduce the level of irreversible damage in MS patients.

In [15]:
async for chunk in after_rag_chain.astream("what are the approches in management of MS"):
    print(chunk, end="", flush=True)

 The context does not provide specific information on the approaches in the management of Multiple Sclerosis (MS). However, it mentions that initial and sustained treatment with high-efficacy therapies is important to optimize patient function for both relapsing and primary progressive MS. Clinical assessments, paraclinical assessments including MRI scans, blood tests, medical history, neurological exams, evoked potentials, cerebrospinal fluid analysis, and optical coherence tomography are used in the diagnosis of MS. Treatment options include interferon β-1a and ocrelizumab for relapsing MS, and possibly other therapies for primary progressive MS. The context also mentions ongoing clinical trials for MS treatments.